In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import zscore

# Chargement des données
data = pd.read_csv("listings.csv")

# Suppression des outliers (z-score > 3 sur 'price')
data = data[(np.abs(zscore(data['price'])) < 3)]

# Log-transformation des colonnes skewées
skewed_cols = ['minimum_nights', 'number_of_reviews', 'reviews_per_month']
for col in skewed_cols:
    if col in data.columns:
        data[col] = data[col].apply(lambda x: np.log1p(x))

# Cible log-transformée
y = np.log1p(data['price'])
X = data.drop(columns=['price'])

# Suppression des colonnes inutiles
for col in ['name', 'last_review', 'host_name', 'neighbourhood_group']:
    if col in X.columns:
        X.drop(columns=col, inplace=True)

# Colonnes numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Prétraitement
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Pipeline complet
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_regression, k='all')),
    ('regressor', SVR())
])

# Grille d'hyperparamètres
param_grid = {
    'regressor__C': [0.01, 0.1, 1, 10, 100],
    'regressor__epsilon': [0.01, 0.1, 0.2, 0.5, 1],
    'regressor__kernel': ['rbf', 'linear', 'poly'],
    'regressor__gamma': ['scale', 'auto']
}

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Validation croisée
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Affichage des résultats de la validation croisée
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values(by="mean_test_score", ascending=False)

print("\nTop 5 des configurations testées :")
print(cv_results[[
    'mean_test_score', 'std_test_score',
    'param_regressor__C',
    'param_regressor__epsilon',
    'param_regressor__kernel',
    'param_regressor__gamma'
]].head(5))

# Meilleur modèle
best_model = grid_search.best_estimator_

# Évaluation
y_pred_log = best_model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_exp = np.expm1(y_test)

mae = mean_absolute_error(y_test_exp, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_exp, y_pred))
r2 = r2_score(y_test_exp, y_pred)

print(f"\nMAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

# Prédiction sur une annonce
sample = X_test.iloc[[0]]
predicted_log_price = best_model.predict(sample)
predicted_price_usd = np.expm1(predicted_log_price[0])
predicted_price_eur = predicted_price_usd * 0.93

print(f"\nPrix prédit pour la nouvelle annonce : {predicted_price_usd:.2f} $ / {predicted_price_eur:.2f} €")



Top 5 des configurations testées :
     mean_test_score  std_test_score  param_regressor__C  \
66         -0.374055        0.005395                 1.0   
60         -0.375257        0.005913                 1.0   
72         -0.375623        0.004504                 1.0   
129        -0.384205        0.006287               100.0   
99         -0.384552        0.004383                10.0   

     param_regressor__epsilon param_regressor__kernel param_regressor__gamma  
66                       0.10                     rbf                  scale  
60                       0.01                     rbf                  scale  
72                       0.20                     rbf                  scale  
129                      0.10                     rbf                   auto  
99                       0.10                     rbf                   auto  

MAE: 53.81
RMSE: 115.87
R²: 0.23

Prix prédit pour la nouvelle annonce : 122.20 $ / 113.65 €


Chargement et nettoyage des données


Lit les données du fichier listings.csv.

Supprime les outliers sur la colonne price en appliquant un z-score > 3.


Applique une transformation logarithmique (log1p) sur les colonnes très asymétriques :
minimum_nights, number_of_reviews, reviews_per_month.



2. Préparation des variables
La variable cible price est également log-transformée (y = log1p(price)).

Supprime les colonnes inutiles ou non pertinentes comme name, host_name, last_review, neighbourhood_group.



3. Prétraitement des données
Crée un pipeline de transformation pour :

les colonnes numériques : imputation (médiane) + standardisation.

les colonnes catégorielles : imputation (valeur fréquente) + encodage OneHot.

Combine tout cela dans un ColumnTransformer.



4. Pipeline complet
Le pipeline applique :

Le prétraitement.

Une sélection de caractéristiques (ici, toutes sont conservées avec k='all').

Un modèle de régression par SVM (SVR).




5. Optimisation par validation croisée
Utilise GridSearchCV avec une grille d’hyperparamètres (C, epsilon, kernel, gamma).

Score utilisé : MAE négatif (pour le minimiser).

Validation croisée avec 3 plis (cv=3).



6. Évaluation finale
Utilise le meilleur modèle trouvé pour prédire les prix sur un jeu de test.

Retransforme les valeurs (expm1) pour revenir aux prix réels en dollars.

Calcule les métriques :

MAE (erreur absolue moyenne),

RMSE (erreur quadratique moyenne),

R² (coefficient de détermination).




7. Prédiction sur une annonce
Prédit le prix d’un logement tiré de l’échantillon test.

Affiche le prix estimé en dollars et en euros (conversion à 0,93 €/USD).

Le meilleur MAE moyen est 0.374 sur les données log-transformées (base e).

L’écart type faible (~0.005) indique une bonne stabilité du modèle durant la validation croisée

Évaluation sur l’ensemble de test (prix réels)



MAE : 53.81 € l'erreur moyenne absolue est d'environ 54 € par annonce.

RMSE : 115.87 €  l'erreur moyenne est plus sévère sur les grandes valeurs.

R² : 0.23 → le modèle n’explique que 23 % de la variance des prix réels.

 Cela montre une perte de performance importante entre validation croisée (sur données transformées) et test final (prix réels)

 Limites du modèle actuel



Perte d’interprétabilité à cause de la transformation logarithmique.

Bien qu’elle stabilise la variance, elle rend les erreurs finales plus difficiles à interpréter directement.

SVM peu performant avec des grands jeux de données hétérogènes :

Surtout avec des variables catégorielles encodées OneHot (augmentation de la dimension).

Temps de calcul élevé pour la grille SVR.

Score R² faible (0.23) :

Le modèle ne capture pas bien les facteurs influents du prix.

Potentiel sous-apprentissage (underfitting).

Pas de traitement des interactions non linéaires entre les variables :

Le modèle RBF peut les détecter, mais pas efficacement avec peu de tuning ou avec beaucoup de bruit.

Données potentiellement déséquilibrées ou bruitées :

Prix Airbnb souvent très variables selon des critères qualitatifs (photos, description, réputation)


 Améliorations possibles


 
Tester d'autres modèles plus robustes :

Random Forest, Gradient Boosting (XGBoost, LightGBM) ou HistGradientBoostingRegressor.

Ces modèles gèrent mieux les interactions et les données hétérogènes.

Réduction de dimension (PCA) ou sélection automatique de variables :

Réduit la complexité et le surapprentissage possible dû à OneHotEncoder.

Feature engineering plus riche :

Créer de nouvelles variables comme : score d’évaluation par nuit, taux d’occupation, etc.

Traitement avancé des catégories :

Utiliser Target Encoding au lieu de OneHot pour les grosses colonnes catégorielles.

Augmenter les folds de validation croisée :

Passer à 5 ou 10 folds pour une meilleure estimation de la variance.

Exploration de modèles basés sur deep learning (si suffisamment de données et de ressources).


 Conclusion


 
Le modèle SVM testé ici, bien que stable lors de la validation croisée, n’offre pas des performances satisfaisantes en production sur des prix Airbnb réels. L’erreur moyenne de 54 € reste élevée, et la faible valeur de R² (0.23) indique que le modèle ne parvient pas à bien modéliser la complexité des prix de location.
Des modèles plus adaptés comme les arbres de décision ensemblistes pourraient fortement améliorer la qualité prédictive tout en conservant une certaine robustesse face aux données bruitées.

